In [ ]:
# pip install pycoingecko

In [ ]:
import pandas as pd
import plotly.express as px
import yfinance as yf
import pandas as pd
import warnings
import logging

warnings.filterwarnings("ignore")
logging.getLogger("yfinance").setLevel(logging.CRITICAL)

In [ ]:
PD_logs_cleaned=pd.read_csv("/content/drive/MyDrive/Post_DC/PD_logs_cleaned.txt",sep="\t")
print(PD_logs_cleaned.shape)
PD_logs_cleaned.head()

(1335, 6)


,channel_id,session_id,coin,exchange,pair,timestamp
0,1095533634,256,APPC,binance,BTC,2019-06-04 12:01:33+00:00
1,1095533634,258,REQ,binance,BTC,2019-06-08 13:00:02+00:00
2,1095533634,278,ADX,binance,BTC,2019-07-05 17:01:12+00:00
3,1095533634,285,SNM,binance,BTC,2019-08-07 16:59:48+00:00
4,1095533634,287,DATA,binance,BTC,2019-08-15 20:00:07+00:00


In [ ]:
PD_logs_cleaned.to_excel("PD_logs_cleaned.xlsx")

In [ ]:
PD_logs_cleaned.drop(columns=['session_id','channel_id'],axis=1,inplace=True)
PD_logs_cleaned = PD_logs_cleaned.drop_duplicates()
PD_logs_cleaned['Month_Year'] = PD_logs_cleaned['timestamp'].str[:10]
PD_logs_cleaned["coin"] = PD_logs_cleaned["coin"] + "-USD"
PD_logs_cleaned['PD']=1
print(PD_logs_cleaned.shape)
PD_logs_cleaned.sort_values(by=['Month_Year']).head()

(1179, 6)


,coin,exchange,pair,timestamp,Month_Year,PD
335,AMB-USD,binance,BTC,2019-01-01 14:30:04+00:00,2019-01-01,1
158,NEBL-USD,binance,BTC,2019-01-03 15:45:32+00:00,2019-01-03,1
345,WTC-USD,yobit,BTC,2019-01-03 17:00:40+00:00,2019-01-03,1
1096,DZC-USD,cryptopia,BTC,2019-01-03 20:10:07+00:00,2019-01-03,1
873,WTC-USD,yobit,BTC,2019-01-03 17:00:24+00:00,2019-01-03,1


In [ ]:
PD_logs_cleaned[(PD_logs_cleaned['coin'] == 'NEBL-USD') & (PD_logs_cleaned['Month_Year'] >= '2022-01-01')]

,coin,exchange,pair,timestamp,Month_Year,PD
62,NEBL-USD,binance,BTC,2022-01-02 17:00:07+00:00,2022-01-02,1
155,NEBL-USD,binance,BTC,2022-01-02 17:00:18+00:00,2022-01-02,1
391,NEBL-USD,binance,BTC,2022-01-02 17:01:23+00:00,2022-01-02,1
700,NEBL-USD,binance,BTC,2022-01-02 17:00:26+00:00,2022-01-02,1
844,NEBL-USD,binance,BTC,2022-01-02 17:02:09+00:00,2022-01-02,1


In [ ]:
print(PD_logs_cleaned['timestamp'].min(),PD_logs_cleaned['timestamp'].max())

2019-01-01 14:30:04+00:00 2022-01-19 18:00:28+00:00


In [ ]:
# PD_logs_cleaned['coin'].unique()

In [ ]:
# PD_logs_cleaned["coin"] = PD_logs_cleaned["coin"] + "-USD"
# (PD_logs_cleaned['coin'] + "-USD").unique().tolist()

In [ ]:
import pandas as pd
import yfinance as yf

start_date = "2019-01-01"
end_date = "2022-01-20"
tickers = PD_logs_cleaned['coin'].unique()

# Initialize an empty list to collect individual ticker data
data_list = []

# Loop through each ticker, download, process, and append its data
for ticker in tickers:
    print("Coin:", ticker)

    # Download historical data for the ticker
    df = yf.download(ticker, start=start_date, end=end_date, interval="1d", group_by="ticker")
    df = df.reset_index()  # Reset index to avoid multi-level indices
    df['coin'] = ticker  # Add a column for the ticker symbol

    # Calculate Market Cap if Close and Volume columns exist
    if 'Close' in df.columns and 'Volume' in df.columns:
        df['Market_Cap'] = df['Close'] * df['Volume']
    else:
        df['Market_Cap'] = pd.NA

    # Calculate Percentage Change if Close column exists and is not fully null
    if 'Close' in df.columns and not df['Close'].isnull().all():
        df['Pct_Change'] = df['Close'].pct_change().fillna(0) * 100
    else:
        df['Pct_Change'] = pd.NA

    # Append the processed DataFrame for this ticker to the list
    data_list.append(df)

# Concatenate all dataframes in the list into a single DataFrame
df_final = pd.concat(data_list, axis=0, ignore_index=True)

# Convert Date to datetime and create Month_Year column
df_final['Date'] = pd.to_datetime(df_final['Date'])
df_final['Month_Year'] = df_final['Date'].dt.to_period('d')

# Display final DataFrame shape and preview
print(df_final.shape)
df_final.head()


Coin: APPC-USD


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Coin: REQ-USD
Coin: ADX-USD


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Coin: SNM-USD
Coin: DATA-USD


...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Coin: CGG-USD
Coin: XTK-USD



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Coin: FLY-USD
Coin: DORA-USD
(182876, 1667)


Ticker                      Date  APPC-USD                                \
Price                                 Open      High       Low     Close   
0      2019-01-01 00:00:00+00:00  0.041474  0.043678  0.040798  0.043616   
1      2019-01-02 00:00:00+00:00  0.043679  0.044965  0.043166  0.044489   
2      2019-01-03 00:00:00+00:00  0.044510  0.044965  0.042623  0.042641   
3      2019-01-04 00:00:00+00:00  0.042639  0.047659  0.042313  0.047206   
4      2019-01-05 00:00:00+00:00  0.047154  0.047764  0.045135  0.045279   

Ticker                          coin Market_Cap Pct_Change  ... FLY-USD  \
Price  Adj Close    Volume                                  ...   Close   
0       0.043616   38159.0  APPC-USD        NaN        NaN  ...     NaN   
1       0.044489  102287.0  APPC-USD        NaN        NaN  ...     NaN   
2       0.042641  138444.0  APPC-USD        NaN        NaN  ...     NaN   
3       0.047206  365047.0  APPC-USD        NaN        NaN  ...     NaN   
4       0.045279  120144.0  APPC-USD        NaN        NaN  ...     NaN   

Ticker                  DORA-USD                                  Month_Year  
Price  Adj Close Volume     Open High Low Close Adj Close Volume              
0            NaN    NaN      NaN  NaN NaN   NaN       NaN    NaN  2019-01-01  
1            NaN    NaN      NaN  NaN NaN   NaN       NaN    NaN  2019-01-02  
2            NaN    NaN      NaN  NaN NaN   NaN       NaN    NaN  2019-01-03  
3            NaN    NaN      NaN  NaN NaN   NaN       NaN    NaN  2019-01-04  
4            NaN    NaN      NaN  NaN NaN   NaN       NaN    NaN  2019-01-05  

[5 rows x 1667 columns]

In [ ]:
import pandas as pd
import yfinance as yf

start_date = "2019-01-01"
end_date = "2022-01-20"
tickers = PD_logs_cleaned['coin'].unique()

df_final = pd.DataFrame()

for ticker in tickers:
    print("Coin : ", ticker)
    df = yf.download(ticker, start=start_date, end=end_date, interval="1d", group_by="ticker")
    df = df.reset_index()
    df['coin'] = ticker
    if 'Close' in df.columns and 'Volume' in df.columns:
        df['Market_Cap'] = df['Close'] * df['Volume']
    else:
        df['Market_Cap'] = pd.NA
    if 'Close' in df.columns and not df['Close'].isnull().all():
      df['Pct_Change'] = df['Close'].pct_change().fillna(0) * 100
    else:
      df['Pct_Change'] = pd.NA
    df_final = pd.concat([df_final, df], ignore_index=True)

df_final['Date'] = pd.to_datetime(df_final['Date'])
df_final['Month_Year'] = df_final['Date'].dt.to_period('d')

print(df_final.shape)
df_final.head()

Coin :  APPC-USD


[*********************100%***********************]  1 of 1 completed


Coin :  REQ-USD


[*********************100%***********************]  1 of 1 completed


Coin :  ADX-USD


...

Coin :  FLY-USD



[*********************100%***********************]  1 of 1 completed

Coin :  DORA-USD


(182876, 1667)


Ticker                      Date  APPC-USD                                \
Price                                 Open      High       Low     Close   
0      2019-01-01 00:00:00+00:00  0.041474  0.043678  0.040798  0.043616   
1      2019-01-02 00:00:00+00:00  0.043679  0.044965  0.043166  0.044489   
2      2019-01-03 00:00:00+00:00  0.044510  0.044965  0.042623  0.042641   
3      2019-01-04 00:00:00+00:00  0.042639  0.047659  0.042313  0.047206   
4      2019-01-05 00:00:00+00:00  0.047154  0.047764  0.045135  0.045279   

Ticker                          coin Market_Cap Pct_Change  ... FLY-USD  \
Price  Adj Close    Volume                                  ...   Close   
0       0.043616   38159.0  APPC-USD        NaN        NaN  ...     NaN   
1       0.044489  102287.0  APPC-USD        NaN        NaN  ...     NaN   
2       0.042641  138444.0  APPC-USD        NaN        NaN  ...     NaN   
3       0.047206  365047.0  APPC-USD        NaN        NaN  ...     NaN   
4       0.045279  120144.0  APPC-USD        NaN        NaN  ...     NaN   

Ticker                  DORA-USD                                  Month_Year  
Price  Adj Close Volume     Open High Low Close Adj Close Volume              
0            NaN    NaN      NaN  NaN NaN   NaN       NaN    NaN  2019-01-01  
1            NaN    NaN      NaN  NaN NaN   NaN       NaN    NaN  2019-01-02  
2            NaN    NaN      NaN  NaN NaN   NaN       NaN    NaN  2019-01-03  
3            NaN    NaN      NaN  NaN NaN   NaN       NaN    NaN  2019-01-04  
4            NaN    NaN      NaN  NaN NaN   NaN       NaN    NaN  2019-01-05  

[5 rows x 1667 columns]

In [ ]:
df_final['coin'].nunique()

220

In [ ]:
print(df_final.shape)
df_final.head()

(182876, 1667)


Ticker                      Date  APPC-USD                                \
Price                                 Open      High       Low     Close   
0      2019-01-01 00:00:00+00:00  0.041474  0.043678  0.040798  0.043616   
1      2019-01-02 00:00:00+00:00  0.043679  0.044965  0.043166  0.044489   
2      2019-01-03 00:00:00+00:00  0.044510  0.044965  0.042623  0.042641   
3      2019-01-04 00:00:00+00:00  0.042639  0.047659  0.042313  0.047206   
4      2019-01-05 00:00:00+00:00  0.047154  0.047764  0.045135  0.045279   

Ticker                          coin Market_Cap Pct_Change  ... FLY-USD  \
Price  Adj Close    Volume                                  ...   Close   
0       0.043616   38159.0  APPC-USD        NaN        NaN  ...     NaN   
1       0.044489  102287.0  APPC-USD        NaN        NaN  ...     NaN   
2       0.042641  138444.0  APPC-USD        NaN        NaN  ...     NaN   
3       0.047206  365047.0  APPC-USD        NaN        NaN  ...     NaN   
4       0.045279  120144.0  APPC-USD        NaN        NaN  ...     NaN   

Ticker                  DORA-USD                                  Month_Year  
Price  Adj Close Volume     Open High Low Close Adj Close Volume              
0            NaN    NaN      NaN  NaN NaN   NaN       NaN    NaN  2019-01-01  
1            NaN    NaN      NaN  NaN NaN   NaN       NaN    NaN  2019-01-02  
2            NaN    NaN      NaN  NaN NaN   NaN       NaN    NaN  2019-01-03  
3            NaN    NaN      NaN  NaN NaN   NaN       NaN    NaN  2019-01-04  
4            NaN    NaN      NaN  NaN NaN   NaN       NaN    NaN  2019-01-05  

[5 rows x 1667 columns]

In [ ]:
print(PD_logs_cleaned.shape)
PD_logs_cleaned.head()

(1179, 6)


,coin,exchange,pair,timestamp,Month_Year,PD
0,APPC-USD,binance,BTC,2019-06-04 12:01:33+00:00,2019-06-04,1
1,REQ-USD,binance,BTC,2019-06-08 13:00:02+00:00,2019-06-08,1
2,ADX-USD,binance,BTC,2019-07-05 17:01:12+00:00,2019-07-05,1
3,SNM-USD,binance,BTC,2019-08-07 16:59:48+00:00,2019-08-07,1
4,DATA-USD,binance,BTC,2019-08-15 20:00:07+00:00,2019-08-15,1


In [ ]:
PD_logs_cleaned.drop(columns=['timestamp'],axis=1,inplace=True)
df_final['Month_Year'] = df_final['Month_Year'].astype(str)
PD_logs_cleaned['Month_Year'] = PD_logs_cleaned['Month_Year'].astype(str)

df_final2=df_final.merge(PD_logs_cleaned[['Month_Year','coin','PD']],how='left')
df_final2['PD'] = df_final2['PD'].fillna(0)
df_final2 = df_final2.drop_duplicates()
print(df_final2.shape)
df_final2.head()

MergeError: Not allowed to merge between different levels. (2 levels on the left, 1 on the right)

In [ ]:
df_final2['PD'].value_counts()

In [ ]:
df_final2['isPenny'] = (df_final2['Open'] < 1).astype(int)
df_final2['drawdown'] = df_final2.groupby('coin')['Open'].transform(lambda x: x.cummax() - x)
df_final2['drawdown_percentage'] = df_final2.groupby('coin')['Open'].transform(lambda x: ((x.cummax() - x) / x.cummax()) * 100)

print(df_final2.shape)
df_final2.head()

In [ ]:
# df_final2[(df_final2['coin']=='APPC-USD') & (df_final2['Month_Year']<='2019-01-31')]

In [ ]:
df_final2=df_final2[df_final2['drawdown_percentage']>=0]

In [ ]:
df_final2['coin'].nunique()

In [ ]:
df_final2.loc[df_final2['Low'] < 0, 'Low'] = (df_final2.loc[df_final2['Low'] < 0, ['High', 'Open']].mean(axis=1))

In [ ]:
df_final2['Month_Year'] = pd.to_datetime(df_final2['Month_Year'])

df_final2 = df_final2.sort_values(by=['coin', 'Month_Year'])

df_final2['unique_index'] = df_final2.groupby('coin').cumcount()

window_sizes = {
    'Open_1m_avg': 30,
    'Open_3m_avg': 90,
    'Open_6m_avg': 180,
    'Open_12m_avg': 360,
    'Open_24m_avg': 720
}

for column, window in window_sizes.items():
    df_final2[column] = (
        df_final2.set_index(['coin', 'unique_index'])['Open']
        .groupby(level=0)
        .rolling(window=window, min_periods=1)
        .mean()
        .reset_index(level=0, drop=True)
        .values
    )

df_final2 = df_final2.drop(columns=['unique_index']).reset_index(drop=True)

print(df_final2.shape)
df_final2.head()

In [ ]:
df_final2['Month_Year'] = pd.to_datetime(df_final2['Month_Year'])

df_final2 = df_final2.sort_values(by=['coin', 'Month_Year'])

df_final2['unique_index'] = df_final2.groupby('coin').cumcount()

window_sizes = {
    'Open_1m_avg': 30,
    'Open_3m_avg': 90,
    'Open_6m_avg': 180,
    'Open_12m_avg': 360,
    'Open_24m_avg': 720
}

for column, window in window_sizes.items():
    df_final2[column] = (
        df_final2.set_index(['coin', 'unique_index'])['Open']
        .groupby(level=0)
        .rolling(window=window, min_periods=1)
        .mean()
        .reset_index(level=0, drop=True)
        .values
    )

for column, window in window_sizes.items():
    std_column_name = column.replace('avg', 'std')
    df_final2[std_column_name] = (
        df_final2.set_index(['coin', 'unique_index'])['Open']
        .groupby(level=0)
        .rolling(window=window, min_periods=1)
        .std()
        .reset_index(level=0, drop=True)
        .values
    )

for column, window in window_sizes.items():
    df_final2['Return'] = df_final2.groupby('coin')['Open'].pct_change()
    vol_column_name = column.replace('avg', 'volatility')
    df_final2[vol_column_name] = (
        df_final2.set_index(['coin', 'unique_index'])['Return']
        .groupby(level=0)
        .rolling(window=window, min_periods=1)
        .std()
        .reset_index(level=0, drop=True)
        .values
    )

df_final2 = df_final2.drop(columns=['unique_index', 'Return']).reset_index(drop=True)

print(df_final2.shape)
df_final2.tail()

In [ ]:
df_final2['market_cap_category'] = pd.cut(df_final2['Market_Cap'], bins=[0, 1e6, 1e9, 1e12], labels=['Micro', 'Small', 'Large'])
print(df_final2.shape)
df_final2.tail()

In [ ]:
#df_final2.to_csv("/content/drive/MyDrive/Post_DC/df.csv")

In [ ]:
df_final2=pd.read_csv("/content/drive/MyDrive/Post_DC/df.csv")
df_final2.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
print(df_final2.shape)
df_final2.head()

(182855, 30)


,Date,Open,High,Low,Close,Adj Close,Volume,coin,Market_Cap,Pct_Change,...,Open_1m_std,Open_3m_std,Open_6m_std,Open_12m_std,Open_24m_std,Open_1m_volatility,Open_3m_volatility,Open_6m_volatility,Open_12m_volatility,Open_24m_volatility
0,2019-01-01,0.002876,0.002930,0.002784,0.002913,0.002913,331289.0,AAC-USD,965.044878,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-02,0.002928,0.002953,0.002757,0.002872,0.002872,305086.0,AAC-USD,876.206987,-1.407486,...,0.000037,0.000037,0.000037,0.000037,0.000037,NaN,NaN,NaN,NaN,NaN
2,2019-01-03,0.002869,0.003170,0.002736,0.002900,0.002900,317624.0,AAC-USD,921.109588,0.974930,...,0.000032,0.000032,0.000032,0.000032,0.000032,0.027033,0.027033,0.027033,0.027033,0.027033
3,2019-01-04,0.002898,0.002986,0.002797,0.002901,0.002901,435891.0,AAC-USD,1264.519778,0.034483,...,0.000027,0.000027,0.000027,0.000027,0.000027,0.020169,0.020169,0.020169,0.020169,0.020169
4,2019-01-05,0.002898,0.003046,0.002845,0.002907,0.002907,518812.0,AAC-USD,1508.186493,0.206827,...,0.000023,0.000023,0.000023,0.000023,0.000023,0.016522,0.016522,0.016522,0.016522,0.016522


In [ ]:
x=df_final2.dtypes
x.to_excel('features.xlsx')